In [4]:
from mostoolkit.ct_utils import resample, crop_to_standard
from mostoolkit.io_utils import sitk_load_with_metadata, sitk_save
from pathlib import Path
import numpy as np
import random

def insight_volume(data, tar):
    # CT org dataset contains images with incorrect pixdim
    imgfn = data
    img, _, spac, _, size = sitk_load_with_metadata(str(imgfn))
    bad_images = [i.name.replace('.png', '.nii.gz') for i in (Path(tar)/'bad_images').glob('*.png')]
    # print(bad_images)
    # print(imgfn.name)
    if spac[0] == 1.0 and spac[1] == 1.0 and spac[2] == 1.0:
        print('(abnormal pixdim){}'.format(Path(imgfn).name))
        return False
    elif imgfn.name in bad_images:
        print('(bad regions){}'.format(Path(imgfn).name))
        return False
    else:
        print('{}'.format(Path(imgfn).name))
        return True
        # return 

ctorg_root = Path(r'D:\Data\ct_data\CT-ORG')
save_path = Path(r'D:\Chang\anaug\data128')
all_ids = np.arange(140)
# first filter the bad images
all_data = [ctorg_root/'volumes'/'volume-{}.nii.gz'.format(i) for i in all_ids]

# filtered_ids = [i for i in all_ids if insight_volume(all_data[i], save_path)]
filtered_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 22, 23, 24, 25, 26, 27, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 134]
print(filtered_ids)

random.Random(123).shuffle(filtered_ids)
train_ids = filtered_ids[:80]
test_ids = filtered_ids[80:]

save_path.mkdir(exist_ok=True)
print(test_ids)
print(ctorg_root)
print(save_path)

train_imgs = [
    [ctorg_root / 'volumes' / f'volume-{i}.nii.gz',
     ctorg_root / 'labels' / f'labels-{i}.nii.gz',
     ] for i in train_ids]

test_imgs = [
    [ctorg_root / 'volumes' / f'volume-{i}.nii.gz',
     ctorg_root / 'labels' / f'labels-{i}.nii.gz',
     ] for i in test_ids]

for tr, trs in train_imgs:
    assert tr.exists() and trs.exists()

for te, tes in test_imgs:
    assert te.exists() and tes.exists()

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 22, 23, 24, 25, 26, 27, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 134]
[110, 10, 123, 100, 129, 79, 2, 117, 55, 120, 121, 119, 21, 24, 134, 67, 66, 95, 92, 72, 6, 15, 130, 76, 122, 13, 58, 8]
D:\Data\ct_data\CT-ORG
D:\Chang\anaug\data128


In [11]:
# preprocess the train images and save to save_path/train
new_space = 3.5     # 128
new_spatial = 128
new_nslices = 128
# new_space = 2.0     # 256
# new_spatial = 160
# new_nslices = 256

for tr, trs in train_imgs+test_imgs:
    print(tr)
    img, d, spac, o, _ = sitk_load_with_metadata(str(tr))
    print(d)
    print(spac)
    print(o)
    break
    seg, _, _, _, _ = sitk_load_with_metadata(str(trs))
    print(o)
    # normalize the voxel values
    img[img<-1024] = -1024
    img[img>4095] = 4095
    img += 1024
    img = resample(img, spac, new_spacing=[new_space, new_space, new_space])
    seg = resample(seg, spac, new_spacing=[new_space, new_space, new_space], order=0)
    # if img.shape[0] < new_nslices:
    # print(img.shape)
    img = crop_to_standard(img, [new_nslices, new_spatial, new_spatial])
    seg = crop_to_standard(seg, [new_nslices, new_spatial, new_spatial])
    # else:
        # img = crop_to_standard(img, [-1, new_spatial, new_spatial])
        # seg = crop_to_standard(seg, [-1, new_spatial, new_spatial])
    sitk_save(str(save_path/ tr.name), img, [new_space,new_space,new_space], np.array(o), np.array(d))
    sitk_save(str(save_path/ trs.name), seg, [new_space,new_space,new_space], np.array(o), np.array(d))
    print(f'{tr.name} done')
    


D:\Data\ct_data\CT-ORG\volumes\volume-26.nii.gz
(1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
(0.86328125, 0.86328125, 0.699999988079071)
(-236.568359375, 87.568359375, -880.0)


In [6]:
# filter and flip the data
import shutil
from pathlib import Path
from mostoolkit.io_utils import sitk_load_with_metadata, sitk_save

# flip_list = [4,13,14,15,16,21,22,26,27,48,49,50,51,52,68]
flip_list = [68]

root = Path(r'D:\Chang\anaug\data128')
save_path = root / 'full'

def flip_volume(vol):
    return vol[:, :, ::-1]

# get full list
full_list = []
for i in (root/'insight'/'full').glob('*.png'):
    full_list.append(i.name.replace('.png', '').replace('volume-', ''))
full_list = [int(i) for i in full_list]

for f in root.glob('volume*.nii.gz'):
    id = f.name.replace('volume-', '').replace('.nii.gz', '')
    id = int(id)
    if id not in full_list:
        print('{} skipped'.format(f.name))
        continue

    if id in flip_list:
        label_fname = str(f).replace('volume', 'labels')
        image, d, s, o, _ = sitk_load_with_metadata(f)
        label = sitk_load_with_metadata(label_fname)[0]
        f_image = flip_volume(image)
        f_label = flip_volume(label)
        sitk_save(save_path/f.name, f_image, s, o, d)
        sitk_save(save_path/Path(label_fname).name, f_label, s,o,d)
        print('{} flipped and saved'.format(f.name))
        # break
    else:
        label_fname = str(f).replace('volume', 'labels')
        shutil.copy(f, save_path/f.name)
        shutil.copy(label_fname, save_path/Path(label_fname).name)
        print('{} copied'.format(f.name))


volume-10.nii.gz skipped
volume-100.nii.gz skipped
volume-101.nii.gz skipped
volume-102.nii.gz skipped
volume-103.nii.gz skipped
volume-104.nii.gz copied
volume-105.nii.gz copied
volume-106.nii.gz copied
volume-107.nii.gz copied
volume-108.nii.gz copied
volume-109.nii.gz copied
volume-11.nii.gz skipped
volume-110.nii.gz copied
volume-111.nii.gz copied
volume-112.nii.gz copied
volume-113.nii.gz copied
volume-114.nii.gz copied
volume-115.nii.gz copied
volume-116.nii.gz copied
volume-117.nii.gz copied
volume-118.nii.gz copied
volume-119.nii.gz copied
volume-12.nii.gz skipped
volume-120.nii.gz copied
volume-121.nii.gz copied
volume-122.nii.gz copied
volume-123.nii.gz copied
volume-124.nii.gz copied
volume-125.nii.gz copied
volume-126.nii.gz copied
volume-127.nii.gz copied
volume-128.nii.gz skipped
volume-129.nii.gz skipped
volume-13.nii.gz flipped and saved
volume-130.nii.gz skipped
volume-134.nii.gz skipped
volume-14.nii.gz flipped and saved
volume-15.nii.gz flipped and saved
volume-16.ni

In [ ]:
# vertical flip volume-68
import shutil
from pathlib import Path
from mostoolkit.io_utils import sitk_load_with_metadata, sitk_save


image, d, s, o, _ = sitk_load_with_metadata(r'D:\Chang\anaug\data128\full\volume-68.nii.gz')
label = sitk_load_with_metadata(r'D:\Chang\anaug\data128\full\labels-68.nii.gz')[0]
image = image[:, :, ::-1]
label = label[:, :, ::-1]
sitk_save(r'D:\Chang\anaug\data128\full\volume-68.nii.gz', image, s, o, d)
sitk_save(r'D:\Chang\anaug\data128\full\labels-68.nii.gz', label, s,o,d)

In [4]:
# we need a dynamic train/val split for further test
import random
from pathlib import Path
from mostoolkit.io_utils import load_json, save_json

rd = random.Random(123)
root = Path(r'D:/Chang/anaug/data128/full')
all_data = list(root.glob(r'volume*.nii.gz'))
print(len(all_data))
rd.shuffle(all_data)
print(len(all_data))

# train 10, val 20
split = {
    'train': [i.name for i in all_data[:10]],
    'val': [i.name for i in all_data[-20:]]
}
save_json(r'D:/Chang/anaug/split_train10.json', split)


# train 20, val 20
split = {
    'train': [i.name for i in all_data[:20]],
    'val': [i.name for i in all_data[-20:]]
}
save_json(r'D:/Chang/anaug/split_train20.json', split)

# val 20
split = {
    'train': [i.name for i in all_data[:-20]],
    'val': [i.name for i in all_data[20:]]
}
save_json(r'D:/Chang/anaug/split_train28.json', split)



48
48


In [1]:
# preprocess of AMOS dataset
from pathlib import Path
import random
from mostoolkit.io_utils import save_json, sitk_load_with_metadata, sitk_save

trainroot = r'D:\Data\AMOS22\AMOS22\amos22\imagesTr'        # 200
valroot = r'D:\Data\AMOS22\AMOS22\amos22\imagesVa'          # 100 
trainroot = Path(trainroot)
valroot = Path(valroot)

save_path = r'D:\Chang\anaug\amos128'

train_data = list(trainroot.glob('*.nii.gz'))
train_data = [str(i) for i in train_data]
train_data = sorted(train_data)[:200]               # the first 200 are ct images

rd = random.Random(123)
rd.shuffle(train_data)

# print(train_data[:3])
train_data = train_data[:40]

test_data = list(valroot.glob('*.nii.gz'))
test_data = [str(i) for i in test_data]

split = {
    'train': [Path(i).name for i in train_data],
    'val': [Path(i).name for i in test_data]
}
save_json(r'D:\Chang\anaug\split_amos_poor.json', split)

In [5]:
from mostoolkit.ct_utils import resample, crop_to_standard
import numpy as np
import shutil
# preprocess the train images and save to save_path/train
new_space = 3.5     # 128
new_spatial = 128
new_nslices = -1
# new_space = 2.0     # 256
# new_spatial = 160
# new_nslices = 256

shutil.rmtree(save_path)
Path(save_path).mkdir(exist_ok=True)

total = len(train_data+test_data)
for n,tr in enumerate(train_data+test_data):
    print(tr)
    trs = tr.replace('images', 'labels')
    img, d, spac, o, _ = sitk_load_with_metadata(str(tr))
    # print(d)
    # print(spac)
    # print(o)
    # break
    seg, _, _, _, _ = sitk_load_with_metadata(str(trs))
    # print(o)
    # normalize the voxel values
    img[img<-1024] = -1024
    img[img>4095] = 4095
    img += 1024
    img = resample(img, spac, new_spacing=[new_space, new_space, new_space])
    seg = resample(seg, spac, new_spacing=[new_space, new_space, new_space], order=0)
    # if img.shape[0] < new_nslices:
    # print(img.shape)
    img = crop_to_standard(img, [new_nslices, new_spatial, new_spatial])
    seg = crop_to_standard(seg, [new_nslices, new_spatial, new_spatial])
    # else:
        # img = crop_to_standard(img, [-1, new_spatial, new_spatial])
        # seg = crop_to_standard(seg, [-1, new_spatial, new_spatial])
    sitk_save(str(Path(save_path)/ Path(tr).name.replace('amos', 'volume')), img, [new_space,new_space,new_space], np.array(o), np.array(d))
    sitk_save(str(Path(save_path)/ Path(trs).name.replace('amos', 'labels')), seg, [new_space,new_space,new_space], np.array(o), np.array(d))
    print(f'[{n}/{total}]{Path(tr).name} done')
    


D:\Data\AMOS22\AMOS22\amos22\imagesTr\amos_0231.nii.gz
[0/300]amos_0231.nii.gz done
D:\Data\AMOS22\AMOS22\amos22\imagesTr\amos_0177.nii.gz
[1/300]amos_0177.nii.gz done
D:\Data\AMOS22\AMOS22\amos22\imagesTr\amos_0025.nii.gz
[2/300]amos_0025.nii.gz done
D:\Data\AMOS22\AMOS22\amos22\imagesTr\amos_0010.nii.gz
[3/300]amos_0010.nii.gz done
D:\Data\AMOS22\AMOS22\amos22\imagesTr\amos_0084.nii.gz
[4/300]amos_0084.nii.gz done
D:\Data\AMOS22\AMOS22\amos22\imagesTr\amos_0098.nii.gz
[5/300]amos_0098.nii.gz done
D:\Data\AMOS22\AMOS22\amos22\imagesTr\amos_0052.nii.gz
[6/300]amos_0052.nii.gz done
D:\Data\AMOS22\AMOS22\amos22\imagesTr\amos_0066.nii.gz
[7/300]amos_0066.nii.gz done
D:\Data\AMOS22\AMOS22\amos22\imagesTr\amos_0336.nii.gz
[8/300]amos_0336.nii.gz done
D:\Data\AMOS22\AMOS22\amos22\imagesTr\amos_0297.nii.gz
[9/300]amos_0297.nii.gz done
D:\Data\AMOS22\AMOS22\amos22\imagesTr\amos_0129.nii.gz
[10/300]amos_0129.nii.gz done
D:\Data\AMOS22\AMOS22\amos22\imagesTr\amos_0381.nii.gz
[11/300]amos_0381.ni

In [18]:
# create amos split for fewer trianing data
from mostoolkit.io_utils import save_json, load_json

all_data_split = load_json(r'D:\Chang\anaug\split_amos.json')
all_data_split.update({'train': all_data_split['train'][:20]})
print(len(all_data_split['train']))
save_json(r'D:\Chang\anaug\split_amos20.json', all_data_split)

20


In [3]:
# preprocess of DECT dataset
from pathlib import Path
import random
from mostoolkit.io_utils import save_json, sitk_load_with_metadata, sitk_save

dataroot = r'D:\Data\ct_data\dect'
labelroot = Path(dataroot) / 'DLMask_9'
imageroot = Path(dataroot) / 'NIIOriginal'

save_path = r'D:\Chang\anaug\dect128'

# find first the images with label
datalist = []
for i in labelroot.glob('*.nii.gz'):
    id = i.name.split('_')[0]
    hect = imageroot/id/f'{id}_1.nii.gz'
    lect = imageroot/id/f'{id}_2.nii.gz'
    if hect.exists() and lect.exists():
        # print(f'{id} complete')
        datalist.append(id)

print(len(datalist))

rd = random.Random(123)
rd.shuffle(datalist)
split = {
    'train': datalist[:22],
    'val': datalist[22:]

}
save_json(r'D:\Chang\anaug\split_dect.json', split)
# trainroot = r'D:\Data\AMOS22\AMOS22\amos22\imagesTr'        # 200
# valroot = r'D:\Data\AMOS22\AMOS22\amos22\imagesVa'          # 100 
# trainroot = Path(trainroot)
# valroot = Path(valroot)

# save_path = r'D:\Chang\anaug\amos128'

# train_data = list(trainroot.glob('*.nii.gz'))
# train_data = [str(i) for i in train_data]
# train_data = sorted(train_data)[:200]               # the first 200 are ct images

# rd = random.Random(123)
# rd.shuffle(train_data)

# # print(train_data[:3])
# train_data = train_data[:40]

# test_data = list(valroot.glob('*.nii.gz'))
# test_data = [str(i) for i in test_data]

# split = {
#     'train': [Path(i).name for i in train_data],
#     'val': [Path(i).name for i in test_data]
# }
# save_json(r'D:\Chang\anaug\split_amos_poor.json', split)

42


In [6]:
from mostoolkit.ct_utils import resample, crop_to_standard
import numpy as np
import shutil
# preprocess the train images and save to save_path/train
new_space = 3.5     # 128
new_spatial = 128
new_nslices = -1
# new_space = 2.0     # 256
# new_spatial = 160
# new_nslices = 256

# shutil.rmtree(save_path)
Path(save_path).mkdir(exist_ok=True)

total = 42
for n,id in enumerate(datalist):
    # print(tr)
    # trs = tr.replace('images', 'labels')
    fnameH = imageroot / id / f'{id}_1.nii.gz'
    fnameL = imageroot / id / f'{id}_2.nii.gz'
    fnamemask = labelroot / f'{id}_Seg.nii.gz'
    imgH, d, spac, o, _ = sitk_load_with_metadata(str(fnameH))
    imgL, _, _, _, _ = sitk_load_with_metadata(str(fnameL))
    seg, _, _, _, _ = sitk_load_with_metadata(str(fnamemask))
    # print(o)
    # normalize the voxel values
    imgH[imgH<-1024] = -1024
    imgH[imgH>4095] = 4095
    imgH += 1024
    imgL[imgL<-1024] = -1024
    imgL[imgL>4095] = 4095
    imgL += 1024
    imgH = resample(imgH, spac, new_spacing=[new_space, new_space, new_space])
    imgL = resample(imgL, spac, new_spacing=[new_space, new_space, new_space])
    seg = resample(seg, spac, new_spacing=[new_space, new_space, new_space], order=0)
    # if img.shape[0] < new_nslices:
    # print(img.shape)
    imgH = crop_to_standard(imgH, [new_nslices, new_spatial, new_spatial])
    imgL = crop_to_standard(imgL, [new_nslices, new_spatial, new_spatial])
    seg = crop_to_standard(seg, [new_nslices, new_spatial, new_spatial])
    # else:
        # img = crop_to_standard(img, [-1, new_spatial, new_spatial])
        # seg = crop_to_standard(seg, [-1, new_spatial, new_spatial])
    savenameH = Path(save_path) / f'volume_{id}_1.nii.gz'
    savenameL = Path(save_path) / f'volume_{id}_2.nii.gz'
    savenameseg = Path(save_path) / f'labels_{id}.nii.gz'
    sitk_save(str(savenameH), imgH, [new_space,new_space,new_space], np.array(o), np.array(d))
    sitk_save(str(savenameL), imgL, [new_space,new_space,new_space], np.array(o), np.array(d))
    sitk_save(str(savenameseg), seg, [new_space,new_space,new_space], np.array(o), np.array(d))
    print(f'[{n}/{total}]{id} done')
    


[0/42]23000SCACWA done
[1/42]23000V3ZRV done
[2/42]23000AIREI done
[3/42]23000HH7LHA done
[4/42]23000RBC3NA done
[5/42]23000SC14G done
[6/42]230005P25H done
[7/42]230009HJXL done
[8/42]230005PO1D done
[9/42]23000L56P9A done
[10/42]23000CCCKG done
[11/42]23000AUKEY done
[12/42]2300019Q51 done
[13/42]23000T1KHL done
[14/42]230006QKM6 done
[15/42]23000WGXK4 done
[16/42]23000KS1KO done
[17/42]23000DXP9T done
[18/42]2300088WW0 done
[19/42]23000DL0D1A done
[20/42]23000UU5MEA done
[21/42]230009HDPL done
[22/42]2300000CO8 done
[23/42]23000RRRB3 done
[24/42]2300044JC8 done
[25/42]23000EMJ6A done
[26/42]23000L5I9T done
[27/42]23000VRG3B done
[28/42]230006EAMQ done
[29/42]230003BG73 done
[30/42]23000XHF9LA done
[31/42]23000ZZVVN done
[32/42]23000SSCWSA done
[33/42]23000EEWAA done
[34/42]23000JFFBJA done
[35/42]230001X35H done
[36/42]2300044EC8 done
[37/42]23000ZJBN7 done
[38/42]23000KS448 done
[39/42]2300044AG0 done
[40/42]23000BB13N done
[41/42]2300033WJF done


In [ ]:
from mostoolkit.io_utils import sitk_load_with_metadata
from pathlib import Path

for f in Path(r'./amos_final40').glob('labels*'):
    final = sitk_load_with_metadata(str(f))[0]
    init = Path(r'./amos_init40') / f.name
    init = sitk_load_with_metadata(str(init))[0]
    print('{} init size: {}, final size: {}'.format(f.name, init.shape, final.shape))